# 03 - Affine Mappings and Polar Cones

This notebook introduces two tools that show up repeatedly in polyhedral reasoning:

- **Affine Mappings:** How polyhedra transform under $y = Dx + d$.
- **Polar Cones:** A dual geometric view of cones

We'll do two things throughout:
1. Derive the formulations
2. Connect them to the corresponding implementations in `src/`

## Affine Mappings

An affine mapping has the form $f(x) = Dx + d$, where $D \in K^{m\times n}$ and $d \in K^n$

If a polyhedron is given as $P = \{x \in K^m \mid Ax \le b\}$, then its *image* $f(P) = \{y \in K^m \mid \exists x \in P:\; y = D x + d\}$ is also a polyhedron.
Have a look [here](https://www2.mathematik.tu-darmstadt.de/~paffenholz/data/preprints/geometry_of_optimization.pdf) for a proof.

**Key idea:** image = projection \
The image can be described by introducing $y$ and projecting out $x$:

$
\begin{aligned}
    Ax & \le b \\
    y - Dx & \le d \\
    -y + Dx & \le -d
\end{aligned}
$

This is where [FME](02-Fourier-Motzkin-Elimination.ipynb) comes into play as a projection algorithm for systems of inequalities.

All the following entities are polyhedra for all $A \in K^{m\times n}$, as they are all affine mappings:

* The *linear hull*: $\quad\operatorname{lin}(A) := \{A \lambda \mid \lambda \in K^n\}$
* The *affine hull*: $\quad\operatorname{aff}(A) := \{A\lambda \mid 1^\top \lambda = 1\}$
* The *conic hull*: $\quad\operatorname{cone}(A) := \{A\lambda \mid \lambda \in K^n,\; \lambda \ge 0\}$
* The *convex hull*: $\quad\operatorname{conv}(A) := \{A\lambda \mid 1^\top \lambda = 1,\; \lambda \ge 0\}$

The same holds for the sum of two polyhedra $P^1$ and $P^2$, which is also an affine mapping.

An important consequence of this is that $\operatorname{conv}(A) + \operatorname{cone}(B)$ is also a polyhedron, where $B$ is given by $\operatorname{cone}(A) = P(B, 0)$.

**Note:** If $D$ is square and invertible, there is a close-form transformation of the inequalities. \
We implement both, the closed-form invertible case and the general projection-based method.


### Implementation

#### Invertible case (closed form)

```{literalinclude} ../src/polyhedra/affine.py
:language: python
:linenos:
:start-after: "# --- AFFINE:image_invertible:start"
:end-before: "# --- AFFINE:image_invertible:end"
```

#### General case (image = projection + FME)

```{literalinclude} ../src/polyhedra/affine.py
:language: python
:linenos:
:start-after: "# --- AFFINE:image_projection:start"
:end-before: "# --- AFFINE:image_projection:end"
```

### Example

In [ ]:
import numpy as np

from src import visualize_polyhedron

# P: a box 0<=x<=2, 0<=y<=1
A = np.array([
    [-1.0,  0.0],
    [ 0.0, -1.0],
    [ 1.0,  0.0],
    [ 0.0,  1.0],
])
b = np.array([0.0, 0.0, 2.0, 1.0]).reshape(-1, 1)

visualize_polyhedron(A, b, xlim=(-1, 4), ylim=(-1, 4), title="Original polyhedron P in x-space")

Now let's apply an affine mapping (rotation by 25° and anisotropic scaling):

In [ ]:
from src.polyhedra.affine import affine_image_hrep_invertible

# Affine map y = D x + d  (invertible D)
theta = np.deg2rad(25)
D = np.array([
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta),  np.cos(theta)],
]) @ np.array([[1.6, 0.0], [0.0, 0.9]])  # rotation + anisotropic scaling

d = np.array([0.8, 0.6])

A_y, b_y = affine_image_hrep_invertible(A, b, D, d)
visualize_polyhedron(A_y, b_y, xlim=(-1, 5), ylim=(-1, 5), title="Image f(P) in y-space")

## Polar Cones

Let $A \in K^{n \times m}$ and let $\operatorname{cone}(A)$ denote the cone generated by the **columns** of $A$:

$\operatorname{cone}(A) := \{A\lambda \mid \lambda \ge 0\}$.


For a set $S \subseteq K^n$, the **polar cone** is given by $S^\circ = \{ y \mid y^\top x \leq 0\;\; \forall x\in S\}$. \
Geometrically, these are all vectors $y$ that form an **obtuse angle**, i.e., $\geq \frac{\pi}{2}$, with all columns of $A$.

A special case is the **orthogonal complement**, given by $S^\perp = \{ y \mid y^\top x = 0 \;\; \forall x \in S\}$ where always $S^\perp \subseteq S^\circ$ \
All these vectors $y$ are forming a perpendicular angle ($= \frac{\pi}{2}$) with the columns.

**Identities:**
1. If $C := \operatorname{cone}(A) = \{A \lambda \mid \lambda \ge 0\}$, then $C^\circ = \{y \mid A^\top y \le 0\}$.
2. If $C := \{x \mid A x \le 0\}$, then $C^\circ = \operatorname{cone}(A^\top) = \{A^\top \lambda \mid \lambda \ge 0\}$.

An important conclusion from these definitions is $\operatorname{cone}(A) = P(A^\top, 0)^\circ$. \
We can further use the [Farkas Lemma](https://en.wikipedia.org/wiki/Farkas%27_lemma) to show that the RHS $b$ of a system $\{x \mid Ax = b, x \ge 0\}$ has to fullfill $b\in \operatorname{cone}(A)$ to be feasible.

### Implementation

```{literalinclude} ../src/polyhedra/polar.py
:language: python
:linenos:
:start-after: "# --- POLAR:from_generators:start"
:end-before: "# --- POLAR:generators_from_hrep:end"
```

### Example

Consider the matrix $A=\begin{pmatrix} -3 & 2\\ 1 & -2 \end{pmatrix}$.


Its **conic hull** is given by the conic hull of its **columns** (generators):


$\operatorname{cone}(A) = \operatorname{cone}(\{\begin{pmatrix}-3\\1\end{pmatrix}, \begin{pmatrix}2\\-2\end{pmatrix}\})$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

A = np.array([
    [-3.0,  2.0],
    [ 1.0, -2.0]
])  # columns are generators a1,a2

a1, a2 = A[:, 0], A[:, 1]

plt.figure()
plt.grid(True)
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.arrow(0, 0, a1[0], a1[1], width=0.02, length_includes_head=True)
plt.arrow(0, 0, a2[0], a2[1], width=0.02, length_includes_head=True)
plt.title("Generators of K = cone(A)")
plt.gca().set_aspect("equal", adjustable="box")
plt.show()

Now let's compute and plot the polar cone (as inequalities):

In [ ]:
from src.polyhedra.polar import polar_cone_hrep_from_generators

A_pol, b_pol = polar_cone_hrep_from_generators(A)
visualize_polyhedron(A_pol, b_pol, xlim=(-5, 5), ylim=(-5, 5), title="Polar cone K° = { y | A^T y <= 0 }")

## Takeaways

- An affine image $f(P) = \{Dx + d \mid x \in P\}$ is a **polyhedron**.
- Conceptual idea: **image = projection** of an extended system in variables $(x,y)$.
- This explains why Fourier–Motzkin elimination is relevant: it is a projection method for inequalities.
- Polar cones provide two geometric views:
  - If $K = \operatorname{cone}(A)$, then $K^\circ = \{y \mid A^\top y \le 0\}$.
  - If $K = \{x \mid Ax \le 0\}$, then $K^\circ = \operatorname{cone}(A^\top)$.
- These identities are the geometric backbone behind many feasibility/certificate theorems (Farkas-type results).

## Next

**Interior versus exterior representations of polyhedra** (H-rep vs. V-rep.) and how to move between them.

➡️ [04 - Interior Exterior Representations](04-Interior-Exterior-Representations.ipynb)